# Выбор локации для скважины

# Содержание:
* Общая информация. Импорт библиотек и данных
* Обучение и проверка модели для каждого региона
* Подготовка к расчету прибыли
* Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели
* Риски и прибыль для каждого региона
* Вывод


## Общая информация и импорт библиотек 


**Цель работы:** Нужно решить, где бурить новую скважину.

**Цель применния**: Проанализирировать возможную прибыль и риски.  

**Исходные данные**:Предоставлены пробы нефти в трёх регионах, характеристики для каждой скважины в регионе известны


## Импорт библиотек


In [1]:
# для работы со структурами данных
import pandas as pd
# разделение датасета на данные для обучения и тестирования
from sklearn.model_selection import train_test_split
# модель
from sklearn.linear_model import LinearRegression
# для расчета  mse
from sklearn.metrics import  mean_squared_error
# для использовоания RandomState
import numpy as np
from numpy.random import RandomState
#для быстрого анализа и визуал.данных
#import sweetviz as sv
import sweetviz as sv
# для выч. коэф.детерминации
from sklearn.metrics import r2_score 
# для доверит. интерв         
from scipy import stats as st


##  Импорт данных 


In [2]:
data_1= pd.read_csv('geo_data_0.csv')
data_2= pd.read_csv('geo_data_1.csv')
data_3= pd.read_csv('geo_data_2.csv')

for i in data_1,data_2,data_3:
    print(i.head(), '\n')

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647 

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305 

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746 



**Существующие столбцы и информация, которую они содержат:**
* id — уникальный идентификатор скважины
* f0, f1, f2 — три признака точек 
* product — объём запасов в скважине (тыс. баррелей)

**Рассмотрим данные геологоразведки более детально:**


In [3]:
# Первый регион
print(data_1.info())
print(data_1.describe())
data_1.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.3437

0

В первом регионе пропуски отсутствуют, признаки имееют одинаковый масштаб данных, дубликатов нет

In [4]:
# Второй регион
print(data_2.info())
print(data_2.describe())
data_2.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.7340

0

Во втором регионе пропуски отсутствуют, признаки имееют одинаковый масштаб данных, дубликатов нет

In [5]:
# Третий регион
print(data_3.info())
print(data_3.describe())
data_3.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.8448

0

Во третьем регионе пропуски отсутствуют, признаки имееют одинаковый масштаб данных, дубликатов нет

In [6]:
report=sv.analyze(data_1)
report.show_html('report1.html')


Report report1.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [7]:
report=sv.analyze(data_2)
report.show_html('report2.html')


Report report2.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [8]:
report=sv.analyze(data_3)
report.show_html('report3.html')


Report report3.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Все признаки имеют распределение схожее с нормальным.Высокая корр между целевым признаком и f2, но так как о признаках ничего не известно, оставим все как есть.

## Обучение и проверка модели для каждого региона 

Создадим две переменные 
* features — запишем в неё признаки
* target —  запишем в нее целевой признак

In [9]:
# признаки для 1 региона
features_1 = data_1.drop(['id','product'] , axis = 1)
target_1 = data_1['product']

# признаки для 2 региона
features_2 = data_2.drop(['id','product'] , axis = 1)
target_2 = data_2['product']

# признаки для 3 региона
features_3 = data_3.drop(['id','product'] , axis = 1)
target_3 = data_3['product']

Разделим данные на обучающую и валидационную выборки в соотношении 75:25:

In [10]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1,
                                                                                        random_state=777,
                                                                                        test_size = 0.25)
print('1 регион')
print('Доля обучающей выборки: {:.0%}'.format(len(features_train_1)/len(data_1)))
print('Количество элементов в обучающей выборке: {}'.format(features_train_1.shape[0]))
print()

                                                                                        
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2,
                                                                                        random_state=777,
                                                                                        test_size = 0.25)   
print('2 регион')
print('Доля обучающей выборки: {:.0%}'.format(len(features_train_2)/len(data_2)))
print('Количество элементов в обучающей выборке: {}'.format(features_train_2.shape[0]))
print()
                                                                                        
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, target_3,
                                                                                        random_state=777,
                                                                                        test_size = 0.25)    
print('3 регион')
print('Доля обучающей выборки: {:.0%}'.format(len(features_train_3)/len(data_3)))
print('Количество элементов в обучающей выборке: {}'.format(features_train_3.shape[0]))

1 регион
Доля обучающей выборки: 75%
Количество элементов в обучающей выборке: 75000

2 регион
Доля обучающей выборки: 75%
Количество элементов в обучающей выборке: 75000

3 регион
Доля обучающей выборки: 75%
Количество элементов в обучающей выборке: 75000


Для расчёта среднего запаса предсказанного сырья и RMSE напишем функцию **lin_model**, в качестве модели используем **линейную регрессию**:

In [11]:
def lin_model(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse =mse**0.5
    return predicted_valid, mse,rmse

In [12]:
predicted_valid_1,mse_1, rmse_1 = lin_model(features_train_1, target_train_1, features_valid_1, target_valid_1)
print('1-ый регион:')
print('Средний запас предсказанного сырья: {:.1f} тыс. баррелей'.format(predicted_valid_1.mean()))
print('MSE: {:.1f}'.format(mse_1))
print('RMSE: {:.1f}'.format(rmse_1))
print("R2:", round(r2_score(target_valid_1, predicted_valid_1),2))
print()

predicted_valid_2,mse_2, rmse_2 = lin_model(features_train_2, target_train_2, features_valid_2, target_valid_2)
print('2-ой регион:')
print('Средний запас предсказанного сырья: {:.2f} тыс. баррелей'.format(predicted_valid_2.mean()))
print('MSE: {:.1f}'.format(mse_2))
print('RMSE: {:.1f}'.format(rmse_2))
print("R2:", r2_score(target_valid_2, predicted_valid_2))
print()


predicted_valid_3,mse_3, rmse_3 = lin_model(features_train_3, target_train_3, features_valid_3, target_valid_3)
print('3-ий регион:')
print('Средний запас предсказанного сырья: {:.1f} тыс. баррелей'.format(predicted_valid_3.mean()))
print('MSE: {:.1f}'.format(mse_3))
print('RMSE: {:.1f}'.format(rmse_3))
print("R2:", round(r2_score(target_valid_3, predicted_valid_3),2))
print()

1-ый регион:
Средний запас предсказанного сырья: 92.7 тыс. баррелей
MSE: 1443.4
RMSE: 38.0
R2: 0.27

2-ой регион:
Средний запас предсказанного сырья: 69.75 тыс. баррелей
MSE: 0.8
RMSE: 0.9
R2: 0.9996232950483743

3-ий регион:
Средний запас предсказанного сырья: 95.0 тыс. баррелей
MSE: 1618.5
RMSE: 40.2
R2: 0.2



### Выводы:
Высокий коэф. детерминации во втором регионе, говорит о том, что модель предсказывает идеально.


## Подготовка к расчету прибыли


* При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для расчёта прибыли.

* Бюджет на разработку скважин в регионе — 10 млрд рублей.

* Один баррель сырья приносит 450 рублей дохода.

* Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%.

* Среди них выбирают регион с наибольшей средней прибылью.

Все ключевые значения для расчётов сохраним в отдельных переменных:

In [13]:
BASE_POINTS= 500
BEST_BASE_POINTS = 200
BUDGET = 10000000000
OIL_COST = 450000
RISK_POINT = 0.025

required_volume = (BUDGET/BEST_BASE_POINTS)/OIL_COST

print('Достаточный объём сырья для безубыточной разработки новой скважины: {:.1f} тыс. баррелей'.format(required_volume))


Достаточный объём сырья для безубыточной разработки новой скважины: 111.1 тыс. баррелей


Достаточный объём сырья для безубыточной разработки новой скважины: 111 тыс. баррелей  

Этот показатель больше средних значений запасов сырья для каждого из трёх регионов.

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

Создадим df и добавим к нему признаки с валид. выборки

In [14]:
data_profit_1 = features_valid_1
data_profit_1.head()

,f0,f1,f2
96275,0.113718,1.088792,1.652847
22712,-0.964788,0.168978,1.157154
31942,1.047647,0.368769,6.921124
87283,2.041288,0.418637,-0.718987
13342,0.510126,0.822430,7.980055


Добавим целевой признак

In [15]:
data_profit_1['product'] = target_valid_1
data_profit_1.head()

<ipython-input-15-a55a509c9801>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_profit_1['product'] = target_valid_1


,f0,f1,f2,product
96275,0.113718,1.088792,1.652847,103.823665
22712,-0.964788,0.168978,1.157154,127.148609
31942,1.047647,0.368769,6.921124,151.007768
87283,2.041288,0.418637,-0.718987,176.320797
13342,0.510126,0.822430,7.980055,109.722362


Добавим предсказанный признак

In [16]:
data_profit_1['product_predict'] = predicted_valid_1
data_profit_1.head()

<ipython-input-16-11c91548f6fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_profit_1['product_predict'] = predicted_valid_1


,f0,f1,f2,product,product_predict
96275,0.113718,1.088792,1.652847,103.823665,73.602223
22712,-0.964788,0.168978,1.157154,127.148609,79.131512
31942,1.047647,0.368769,6.921124,151.007768,122.033246
87283,2.041288,0.418637,-0.718987,176.320797,74.697464
13342,0.510126,0.822430,7.980055,109.722362,120.607144


Аналогично для 2 и 3 региона

In [17]:
data_profit_2 = features_valid_2
data_profit_2['product'] = target_valid_2
data_profit_2['product_predict'] = predicted_valid_2

data_profit_3 = features_valid_3
data_profit_3['product'] = target_valid_3
data_profit_3['product_predict'] = predicted_valid_3

<ipython-input-17-880de282fa66>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_profit_2['product'] = target_valid_2
<ipython-input-17-880de282fa66>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_profit_2['product_predict'] = predicted_valid_2
<ipython-input-17-880de282fa66>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [18]:
def func_profit(data):
    data.sample(500,replace=True)
    data_list200 =(data.sort_values(by='product_predict', ascending=False)).head(200)
    profit = (data_list200['product'].sum())*OIL_COST - BUDGET
    return round(profit,2)

In [19]:
print("Прибыль для 1 регоина по выбранным скважинам:",func_profit(data_profit_1))
print("Прибыль для 2 регоина по выбранным скважинам:",func_profit(data_profit_2))
print("Прибыль для 3 регоина по выбранным скважинам:",func_profit(data_profit_3))

Прибыль для 1 регоина по выбранным скважинам: 3428943543.81
Прибыль для 2 регоина по выбранным скважинам: 2415086696.68
Прибыль для 3 регоина по выбранным скважинам: 2272190444.81


## Риски и прибыль для каждого региона

In [20]:
state = np.random.RandomState(12345)
values_1 = []

#Процедурой бутстреп создаем 1000 подвыборок
for i in range(1000):
    subsample = data_profit_1.sample(n=500, replace=True, random_state=state)
    values_1.append(func_profit(subsample))
values_1 = pd.Series(values_1)

In [21]:
lower = values_1.quantile(0.025)
upper = values_1.quantile(0.975)
print('Доверительный интервал 95% от {:.2f} до {:.2f}'.format(lower, upper))

Доверительный интервал 95% от -44989841.57 до 1011837967.13


In [22]:
print('Средняя прибыль: {:.2f} руб.'.format(values_1.mean()))

Средняя прибыль: 457865193.14 руб.


In [23]:
print('Предполагаемый риск: {:.2%}'.format((values_1 < 0).mean()))

Предполагаемый риск: 3.90%


Аналогично для 2 региона

In [24]:
state = np.random.RandomState(12345)
values_2 = []

#Процедурой бутстреп создаем 1000 подвыборок
for i in range(1000):
    subsample = data_profit_2.sample(n=500, replace=True, random_state=state)
    values_2.append(func_profit(subsample))
values_2 = pd.Series(values_2)
lower = values_2.quantile(0.025)
upper = values_2.quantile(0.975)
print('Доверительный интервал 95% от {:.2f} до {:.2f}'.format(lower, upper))
print('Средняя прибыль: {:.2f} руб.'.format(values_2.mean()))
print('Предполагаемый риск: {:.2%}'.format((values_2 < 0).mean()))


Доверительный интервал 95% от 135221787.40 до 899393387.51
Средняя прибыль: 520072307.83 руб.
Предполагаемый риск: 0.40%


Для 3-его региона

In [25]:
values_3 = []

#Процедурой бутстреп создаем 1000 подвыборок
for i in range(1000):
    subsample = data_profit_3.sample(n=500, replace=True, random_state=state)
    values_3.append(func_profit(subsample))
values_3 = pd.Series(values_3)
lower = values_3.quantile(0.025)
upper = values_3.quantile(0.975)
print('Доверительный интервал 95% от {:.2f} до {:.2f}'.format(lower, upper))
print('Средняя прибыль: {:.2f} руб.'.format(values_3.mean()))
print('Предполагаемый риск: {:.2%}'.format((values_3 < 0).mean()))

Доверительный интервал 95% от -184291547.44 до 862003926.64
Средняя прибыль: 334265276.34 руб.
Предполагаемый риск: 11.40%



## Вывод


Наилучшее место- 2 регион: прибыль больше чем в других регоинах,риск убытков меньше чем в других и меньше установленного 2.5%